In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client
import pandas as pd
from glob import glob as gb

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata

session = cc.database.create_session()

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:39417 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
input_path = '/g/data/e14/erd561/ARGO/'

output_path = '/g/data/e14/erd561/Australia_3/'

In [10]:
lonW = 150
lonE = 182

latL = -32.5

temp_ym32_LTM = xr.open_dataset(input_path + 'RG_ArgoClim_Temperature_2017.nc', decode_times=False).\
                ARGO_TEMPERATURE_MEAN.\
                sel(LONGITUDE=slice(lonW,lonE)).\
                sel(LATITUDE=latL)
print(temp_ym32_LTM)

<xarray.DataArray 'ARGO_TEMPERATURE_MEAN' (PRESSURE: 58, LONGITUDE: 32)>
array([[   nan,    nan, 22.29 , ..., 19.52 , 19.456, 19.348],
       [   nan,    nan, 22.164, ..., 19.49 , 19.421, 19.313],
       [   nan,    nan, 21.873, ..., 19.404, 19.323, 19.213],
       ...,
       [   nan,    nan,  2.489, ...,  2.592,  2.589,  2.584],
       [   nan,    nan,  2.398, ...,  2.483,  2.492,  2.492],
       [   nan,    nan,  2.348, ...,  2.414,  2.425,  2.429]], dtype=float32)
Coordinates:
  * LONGITUDE  (LONGITUDE) float32 150.5 151.5 152.5 153.5 ... 179.5 180.5 181.5
    LATITUDE   float32 -32.5
  * PRESSURE   (PRESSURE) float32 2.5 10.0 20.0 30.0 ... 1800.0 1900.0 1975.0
Attributes:
    units:      degree celcius (ITS-90)
    long_name:  ARGO TEMPERATURE MEAN Jan 2004 - Dec 2016 (13.0 year) RG CLIM...


In [11]:
temp_ym32_months = xr.open_dataset(input_path + 'RG_ArgoClim_Temperature_2017.nc', decode_times=False).\
                ARGO_TEMPERATURE_ANOMALY.\
                sel(LONGITUDE=slice(lonW,lonE)).\
                sel(LATITUDE=latL)
print(temp_ym32_months)

TIME = [pd.date_range('15/1/2004',periods=156, freq=pd.DateOffset(months=1))]
print(TIME)

temp_ym32_months = temp_ym32_months.assign_coords(TIME=np.squeeze(TIME))
print(temp_ym32_months)

temp_ym32 = temp_ym32_months.groupby('TIME.year').mean('TIME', skipna=True)
print(temp_ym32)

temp_ym32_2004_2016 = temp_ym32.sel(year=slice(1993,2017))
print(temp_ym32_2004_2016)

<xarray.DataArray 'ARGO_TEMPERATURE_ANOMALY' (TIME: 156, PRESSURE: 58, LONGITUDE: 32)>
[289536 values with dtype=float32]
Coordinates:
  * LONGITUDE  (LONGITUDE) float32 150.5 151.5 152.5 153.5 ... 179.5 180.5 181.5
    LATITUDE   float32 -32.5
  * PRESSURE   (PRESSURE) float32 2.5 10.0 20.0 30.0 ... 1800.0 1900.0 1975.0
  * TIME       (TIME) float32 0.5 1.5 2.5 3.5 4.5 ... 152.5 153.5 154.5 155.5
Attributes:
    units:      degree celcius (ITS-90)
    long_name:  ARGO TEMPERATURE ANOMALY defined by Jan 2004 - Dec 2016 (13.0...
[DatetimeIndex(['2004-01-15', '2004-02-15', '2004-03-15', '2004-04-15',
               '2004-05-15', '2004-06-15', '2004-07-15', '2004-08-15',
               '2004-09-15', '2004-10-15',
               ...
               '2016-03-15', '2016-04-15', '2016-05-15', '2016-06-15',
               '2016-07-15', '2016-08-15', '2016-09-15', '2016-10-15',
               '2016-11-15', '2016-12-15'],
              dtype='datetime64[ns]', length=156, freq='<DateOffset: months

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [12]:
temp_ym32_2017 = xr.open_mfdataset(sorted(gb(input_path + 'RG_ArgoClim_2017*.nc')), decode_times=False).\
                ARGO_TEMPERATURE_ANOMALY.\
                sel(LONGITUDE=slice(lonW,lonE)).\
                sel(LATITUDE=latL).\
                assign_coords(TIME=np.squeeze([pd.date_range('15/1/2017',periods=12, freq=pd.DateOffset(months=1))])).\
                groupby('TIME.year').mean('TIME', skipna=True)
temp_ym32_2017.load()
print(temp_ym32_2017)

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  """Entry point for launching an IPython kernel.
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


<xarray.DataArray 'ARGO_TEMPERATURE_ANOMALY' (year: 1, PRESSURE: 58, LONGITUDE: 32)>
array([[[           nan,            nan,  3.1649998e-01, ...,
          3.0624995e-01,  3.2591665e-01,  4.1633335e-01],
        [           nan,            nan,  3.1408340e-01, ...,
          3.0349994e-01,  3.1591654e-01,  3.8283339e-01],
        [           nan,            nan,  3.1283334e-01, ...,
          3.1083339e-01,  3.1108323e-01,  3.7474999e-01],
        ...,
        [           nan,            nan,  2.4583334e-02, ...,
          1.7500002e-03,  2.4999999e-03, -6.6666660e-04],
        [           nan,            nan,  1.8416665e-02, ...,
         -2.0833332e-03, -1.6666669e-04, -2.4999999e-03],
        [           nan,            nan,  6.7500006e-03, ...,
         -5.6666671e-03, -1.6666665e-03,  6.6666660e-04]]], dtype=float32)
Coordinates:
  * LONGITUDE  (LONGITUDE) float32 150.5 151.5 152.5 153.5 ... 179.5 180.5 181.5
  * PRESSURE   (PRESSURE) float32 2.5 10.0 20.0 30.0 ... 1800.0 1900.0 

In [31]:
temp_ym32 = xr.concat([temp_ym32_2004_2016, temp_ym32_2017], dim='year') + temp_ym32_LTM
print(temp_ym32)

temp_ym32 = temp_ym32.assign_coords(PRESSURE=gsw.z_from_p(temp_ym32.PRESSURE, temp_ym32.LATITUDE))
print(temp_ym32)

<xarray.DataArray (year: 14, PRESSURE: 58, LONGITUDE: 32)>
array([[[       nan,        nan, 22.167917 , ..., 19.375    ,
         19.330332 , 19.228249 ],
        [       nan,        nan, 22.044582 , ..., 19.347    ,
         19.298082 , 19.19525  ],
        [       nan,        nan, 21.75825  , ..., 19.267666 ,
         19.2055   , 19.0975   ],
        ...,
        [       nan,        nan,  2.4855833, ...,  2.5909166,
          2.58775  ,  2.58275  ],
        [       nan,        nan,  2.3935   , ...,  2.481    ,
          2.4900835,  2.4905002],
        [       nan,        nan,  2.3444166, ...,  2.4103334,
          2.4216666,  2.4263332]],

       [[       nan,        nan, 22.332167 , ..., 19.438583 ,
         19.376749 , 19.242167 ],
        [       nan,        nan, 22.206165 , ..., 19.409332 ,
         19.342916 , 19.207666 ],
        [       nan,        nan, 21.914333 , ..., 19.326332 ,
         19.247583 , 19.106833 ],
        ...,
        [       nan,        nan,  2.4885833, ...,

In [36]:
year = temp_ym32.year
depth = temp_ym32.PRESSURE
lon = temp_ym32.LONGITUDE

# temp_ym32_trans = np.transpose(temp_ym32, (2,1,0))
temp_ym32_trans = temp_ym32
print(np.shape(temp_ym32_trans))
temp_ym32_slope = xr.DataArray(
    np.zeros([np.shape(depth)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[depth, lon], name='temp_ym32')
temp_ym32_p_value = xr.DataArray(
    np.zeros([np.shape(depth)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[depth, lon], name='temp_ym32')
temp_ym32_std_err = xr.DataArray(
    np.zeros([np.shape(depth)[0], np.shape(lon)[0]]), dims=(
    'depth', 'lon'), coords=[depth, lon], name='temp_ym32')
for iid, i in enumerate(depth):
    for jid, j in enumerate(lon):
        temp_ym32_slope[iid, jid], _, _, temp_ym32_p_value[iid, jid], temp_ym32_std_err[iid, jid] = \
        stats.linregress(year, temp_ym32_trans[:, iid, jid])
    print('depth ' + str(np.array(i)))
print(temp_ym32_slope)

(14, 58, 32)
depth -2.4824667902378463
depth -9.929686323832927
depth -19.858890502154814
depth -29.78761268374492
depth -39.71585301734021
depth -49.64361165163537
depth -59.570888735282736
depth -69.4976844168924
depth -79.42399884503213
depth -89.34983216822738
depth -99.27518453496121
depth -109.20005609367445
depth -119.12444699276553
depth -129.0483573805905
depth -138.97178740546315
depth -148.89473721565483
depth -158.81720695939458
depth -168.739196784869
depth -181.14100940733599
depth -198.5022882329311
depth -218.34195232182697
depth -238.17970029053743
depth -258.0155333220116
depth -277.8494525985191
depth -297.6814593016509
depth -317.5115546123179
depth -337.33973971075176
depth -357.16601577650414
depth -376.99038398844647
depth -396.8128455247699
depth -416.63340156298517
depth -436.45205327992164
depth -458.745761666863
depth -495.89659426113866
depth -545.4206505050981
depth -594.9328524440836
depth -644.4332183781514
depth -693.9217665806705
depth -743.398515298296

In [59]:
# literally copy-pasted from 
# /g/data/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_B1/output059/ocean

depth_1 = -xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_B1/output059/ocean/ocean.nc').\
st_ocean.sel(st_ocean=slice(0,2100)).values

print(depth_1)

lon_1 = xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2/1deg_jra55v13_iaf_spinup1_B1/output059/ocean/ocean.nc').\
xt_ocean.sel(xt_ocean=slice(lonW-360,lonE-360)).values +360

print(lon_1)

grid_lon_1, grid_depth_1 = np.meshgrid(lon_1,depth_1)

temp_ym32_grid_lon, temp_ym32_grid_depth = np.meshgrid(temp_ym32.LONGITUDE, temp_ym32.PRESSURE)
temp_ym32_lon_depth = np.transpose(temp_ym32_grid_lon.flatten('F'))
temp_ym32_depth_depth = temp_ym32_grid_depth.flatten('F')
temp_ym32_fdepth_list = [temp_ym32_lon_fdepth, temp_ym32_depth_fdepth]    
temp_ym32_grid_points = np.transpose(temp_ym32_fdepth_list)

for y in year:
    temp_ym32_flatten = temp_ym32.sel(year=y).stack(z=('LONGITUDE', 'PRESSURE'))

    temp_ym32_1_now = interpolate.griddata(temp_ym32_grid_points, temp_ym32_flatten, 
                                 (grid_lon_1, grid_depth_1), 
                                 method='linear')
    
    if y == 2004:
        temp_ym32_1 = temp_ym32_1_now
    else:
        temp_ym32_1 = np.dstack((temp_ym32_1, temp_ym32_1_now))
        
    print(y)

print(np.shape(temp_ym32_1))

[-1.15174989e+00 -3.64867412e+00 -6.56491847e+00 -9.97086919e+00
 -1.39487084e+01 -1.85943795e+01 -2.40198736e+01 -3.03558852e+01
 -3.77548899e+01 -4.63947019e+01 -5.64825697e+01 -6.82598648e+01
 -8.20074097e+01 -9.80514644e+01 -1.16770349e+02 -1.38601601e+02
 -1.64049432e+02 -1.93692057e+02 -2.28188144e+02 -2.68281214e+02
 -3.14800182e+02 -3.68653547e+02 -4.30813944e+02 -5.02289293e+02
 -5.84077100e+02 -6.77100253e+02 -7.82126722e+02 -8.99681777e+02
 -1.02996811e+03 -1.17281337e+03 -1.32766203e+03 -1.49361845e+03
 -1.66953340e+03 -1.85411412e+03]
[150.5 151.5 152.5 153.5 154.5 155.5 156.5 157.5 158.5 159.5 160.5 161.5
 162.5 163.5 164.5 165.5 166.5 167.5 168.5 169.5 170.5 171.5 172.5 173.5
 174.5 175.5 176.5 177.5 178.5 179.5 180.5 181.5]
<xarray.DataArray 'year' ()>
array(2004)
Coordinates:
    LATITUDE  float32 -32.5
    year      int64 2004
<xarray.DataArray 'year' ()>
array(2005)
Coordinates:
    LATITUDE  float32 -32.5
    year      int64 2005
<xarray.DataArray 'year' ()>
array(2

In [65]:
temp_ym32_1_trans = np.transpose(temp_ym32_1, (2,1,0))
print(np.shape(temp_ym32_1_trans))
temp_ym32_1_slope = xr.DataArray(
    np.zeros([np.shape(depth_1)[0], np.shape(lon_1)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1, lon_1], name='temp_ym32_1')
temp_ym32_1_p_value = xr.DataArray(
    np.zeros([np.shape(depth_1)[0], np.shape(lon_1)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1, lon_1], name='temp_ym32')
temp_ym32_1_std_err = xr.DataArray(
    np.zeros([np.shape(depth_1)[0], np.shape(lon_1)[0]]), dims=(
    'depth_1', 'lon_1'), coords=[depth_1, lon_1], name='temp_ym32')
for iid, i in enumerate(depth_1):
    for jid, j in enumerate(lon_1):
        temp_ym32_1_slope[iid, jid], _, _, temp_ym32_1_p_value[iid, jid], temp_ym32_1_std_err[iid, jid] = \
        stats.linregress(year, temp_ym32_1_trans[:, jid, iid])
    print('depth_1 ' + str(np.array(i)))
print(temp_ym32_1_slope)

(14, 32, 34)
depth_1 -1.15174989182586
depth_1 -3.6486741197120196
depth_1 -6.564918471676153
depth_1 -9.970869194872916
depth_1 -13.948708373749158
depth_1 -18.594379460593196
depth_1 -24.019873617545198
depth_1 -30.355885242559452
depth_1 -37.754889932903495
depth_1 -46.39470194845016
depth_1 -56.48256969937929
depth_1 -68.25986482598941
depth_1 -82.00740970018322
depth_1 -98.05146439730592
depth_1 -116.77034928747688
depth_1 -138.60160118643546
depth_1 -164.04943216631892
depth_1 -193.69205664349184
depth_1 -228.1881444592732
depth_1 -268.2812141854562
depth_1 -314.800181990729
depth_1 -368.65354740782504
depth_1 -430.8139438640983
depth_1 -502.28929308570173
depth_1 -584.0771003035616
depth_1 -677.1002526914099
depth_1 -782.1267221440287
depth_1 -899.6817765561261
depth_1 -1029.9681126743835
depth_1 -1172.813371535859
depth_1 -1327.6620295174746
depth_1 -1493.618454761941
depth_1 -1669.533396965427
depth_1 -1854.1141182462907
<xarray.DataArray 'temp_ym32_1' (depth_1: 34, lon_1: 32)

In [71]:
# literally copy-pasted from 
# /g/data/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_iaf_gmredi6/output153/ocean

depth_025 = -xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_iaf_gmredi6/output153/ocean/ocean.nc').\
st_ocean.sel(st_ocean=slice(0,2000)).values

lon_025 = xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2-025/025deg_jra55v13_iaf_gmredi6/output153/ocean/ocean.nc').\
xt_ocean.sel(xt_ocean=slice(lonW-360,lonE-360)).values +360

print(depth_025)

print(lon_025)

grid_lon_025, grid_depth_025 = np.meshgrid(lon_025,depth_025)

temp_ym32_grid_lon, temp_ym32_grid_depth = np.meshgrid(temp_ym32.LONGITUDE, temp_ym32.PRESSURE)
temp_ym32_lon_depth = np.transpose(temp_ym32_grid_lon.flatten('F'))
temp_ym32_depth_depth = temp_ym32_grid_depth.flatten('F')
temp_ym32_fdepth_list = [temp_ym32_lon_fdepth, temp_ym32_depth_fdepth]    
temp_ym32_grid_points = np.transpose(temp_ym32_fdepth_list)

for y in year:
    temp_ym32_flatten = temp_ym32.sel(year=y).stack(z=('LONGITUDE', 'PRESSURE'))

    temp_ym32_025_now = interpolate.griddata(temp_ym32_grid_points, temp_ym32_flatten, 
                                 (grid_lon_025, grid_depth_025), 
                                 method='linear')
    
    if y == 2004:
        temp_ym32_025 = temp_ym32_025_now
    else:
        temp_ym32_025 = np.dstack((temp_ym32_025, temp_ym32_025_now))
        
    print(y)

print(np.shape(temp_ym32_025))

[-1.15174989e+00 -3.64867412e+00 -6.56491847e+00 -9.97086919e+00
 -1.39487084e+01 -1.85943795e+01 -2.40198736e+01 -3.03558852e+01
 -3.77548899e+01 -4.63947019e+01 -5.64825697e+01 -6.82598648e+01
 -8.20074097e+01 -9.80514644e+01 -1.16770349e+02 -1.38601601e+02
 -1.64049432e+02 -1.93692057e+02 -2.28188144e+02 -2.68281214e+02
 -3.14800182e+02 -3.68653547e+02 -4.30813944e+02 -5.02289293e+02
 -5.84077100e+02 -6.77100253e+02 -7.82126722e+02 -8.99681777e+02
 -1.02996811e+03 -1.17281337e+03 -1.32766203e+03 -1.49361845e+03
 -1.66953340e+03 -1.85411412e+03]
[150.125 150.375 150.625 150.875 151.125 151.375 151.625 151.875 152.125
 152.375 152.625 152.875 153.125 153.375 153.625 153.875 154.125 154.375
 154.625 154.875 155.125 155.375 155.625 155.875 156.125 156.375 156.625
 156.875 157.125 157.375 157.625 157.875 158.125 158.375 158.625 158.875
 159.125 159.375 159.625 159.875 160.125 160.375 160.625 160.875 161.125
 161.375 161.625 161.875 162.125 162.375 162.625 162.875 163.125 163.375
 163.625

In [72]:
temp_ym32_025_trans = np.transpose(temp_ym32_025, (2,1,0))
print(np.shape(temp_ym32_025_trans))
temp_ym32_025_slope = xr.DataArray(
    np.zeros([np.shape(depth_025)[0], np.shape(lon_025)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025, lon_025], name='temp_ym32_025')
temp_ym32_025_p_value = xr.DataArray(
    np.zeros([np.shape(depth_025)[0], np.shape(lon_025)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025, lon_025], name='temp_ym32')
temp_ym32_025_std_err = xr.DataArray(
    np.zeros([np.shape(depth_025)[0], np.shape(lon_025)[0]]), dims=(
    'depth_025', 'lon_025'), coords=[depth_025, lon_025], name='temp_ym32')
for iid, i in enumerate(depth_025):
    for jid, j in enumerate(lon_025):
        temp_ym32_025_slope[iid, jid], _, _, temp_ym32_025_p_value[iid, jid], temp_ym32_025_std_err[iid, jid] = \
        stats.linregress(year, temp_ym32_025_trans[:, jid, iid])
    print('depth_025 ' + str(np.array(i)))
print(temp_ym32_025_slope)

(14, 128, 34)
depth_025 -1.15174989182586
depth_025 -3.6486741197120196
depth_025 -6.564918471676153
depth_025 -9.970869194872916
depth_025 -13.948708373749158
depth_025 -18.594379460593196
depth_025 -24.019873617545198
depth_025 -30.355885242559452
depth_025 -37.754889932903495
depth_025 -46.39470194845016
depth_025 -56.48256969937929
depth_025 -68.25986482598941
depth_025 -82.00740970018322
depth_025 -98.05146439730592
depth_025 -116.77034928747688
depth_025 -138.60160118643546
depth_025 -164.04943216631892
depth_025 -193.69205664349184
depth_025 -228.1881444592732
depth_025 -268.2812141854562
depth_025 -314.800181990729
depth_025 -368.65354740782504
depth_025 -430.8139438640983
depth_025 -502.28929308570173
depth_025 -584.0771003035616
depth_025 -677.1002526914099
depth_025 -782.1267221440287
depth_025 -899.6817765561261
depth_025 -1029.9681126743835
depth_025 -1172.813371535859
depth_025 -1327.6620295174746
depth_025 -1493.618454761941
depth_025 -1669.533396965427
depth_025 -1854.1

In [74]:
# literally copy-pasted from 
# /g/data/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_iaf/output197/ocean/ocean.nc

depth_01 = -xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_iaf/output197/ocean/ocean.nc').\
st_ocean.sel(st_ocean=slice(0,2000)).values

lon_01 = xr.open_dataset(
'/g/data/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_iaf/output197/ocean/ocean.nc').\
xt_ocean.sel(xt_ocean=slice(lonW-360,lonE-360)).values +360

print(depth_01)

print(lon_01)

grid_lon_01, grid_depth_01 = np.meshgrid(lon_01,depth_01)

temp_ym32_grid_lon, temp_ym32_grid_depth = np.meshgrid(temp_ym32.LONGITUDE, temp_ym32.PRESSURE)
temp_ym32_lon_depth = np.transpose(temp_ym32_grid_lon.flatten('F'))
temp_ym32_depth_depth = temp_ym32_grid_depth.flatten('F')
temp_ym32_fdepth_list = [temp_ym32_lon_fdepth, temp_ym32_depth_fdepth]    
temp_ym32_grid_points = np.transpose(temp_ym32_fdepth_list)

for y in year:
    temp_ym32_flatten = temp_ym32.sel(year=y).stack(z=('LONGITUDE', 'PRESSURE'))

    temp_ym32_01_now = interpolate.griddata(temp_ym32_grid_points, temp_ym32_flatten, 
                                 (grid_lon_01, grid_depth_01), 
                                 method='linear')
    
    if y == 2004:
        temp_ym32_01 = temp_ym32_01_now
    else:
        temp_ym32_01 = np.dstack((temp_ym32_01, temp_ym32_01_now))
        
    print(y)

print(np.shape(temp_ym32_01))

[-5.41280765e-01 -1.68073468e+00 -2.93995265e+00 -4.33152149e+00
 -5.86935042e+00 -7.56880992e+00 -9.44688496e+00 -1.15223444e+01
 -1.38159279e+01 -1.63505526e+01 -1.91515408e+01 -2.22468718e+01
 -2.56674591e+01 -2.94474571e+01 -3.36245984e+01 -3.82405659e+01
 -4.33414022e+01 -4.89779610e+01 -5.52064020e+01 -6.20887357e+01
 -6.96934198e+01 -7.80960121e+01 -8.73798840e+01 -9.76369969e+01
 -1.08968745e+02 -1.21486866e+02 -1.35314420e+02 -1.50586834e+02
 -1.67453006e+02 -1.86076460e+02 -2.06636535e+02 -2.29329581e+02
 -2.54370129e+02 -2.81991982e+02 -3.12449164e+02 -3.46016618e+02
 -3.82990565e+02 -4.23688336e+02 -4.68447515e+02 -5.17624173e+02
 -5.71589915e+02 -6.30727513e+02 -6.95424821e+02 -7.66066799e+02
 -8.43025513e+02 -9.26648199e+02 -1.01724371e+03 -1.11506797e+03
 -1.22030950e+03 -1.33307609e+03 -1.45338444e+03 -1.58115374e+03
 -1.71620465e+03 -1.85826403e+03]
[150.05 150.15 150.25 150.35 150.45 150.55 150.65 150.75 150.85 150.95
 151.05 151.15 151.25 151.35 151.45 151.55 151.65 

In [75]:
temp_ym32_01_trans = np.transpose(temp_ym32_01, (2,1,0))
print(np.shape(temp_ym32_01_trans))
temp_ym32_01_slope = xr.DataArray(
    np.zeros([np.shape(depth_01)[0], np.shape(lon_01)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01, lon_01], name='temp_ym32_01')
temp_ym32_01_p_value = xr.DataArray(
    np.zeros([np.shape(depth_01)[0], np.shape(lon_01)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01, lon_01], name='temp_ym32')
temp_ym32_01_std_err = xr.DataArray(
    np.zeros([np.shape(depth_01)[0], np.shape(lon_01)[0]]), dims=(
    'depth_01', 'lon_01'), coords=[depth_01, lon_01], name='temp_ym32')
for iid, i in enumerate(depth_01):
    for jid, j in enumerate(lon_01):
        temp_ym32_01_slope[iid, jid], _, _, temp_ym32_01_p_value[iid, jid], temp_ym32_01_std_err[iid, jid] = \
        stats.linregress(year, temp_ym32_01_trans[:, jid, iid])
    print('depth_01 ' + str(np.array(i)))
print(temp_ym32_01_slope)

(14, 320, 54)
depth_01 -0.5412807653916101
depth_01 -1.680734679831433
depth_01 -2.939952648914465
depth_01 -4.331521485149508
depth_01 -5.8693504240540255
depth_01 -7.5688099200502155
depth_01 -9.446884959648713
depth_01 -11.522344392803275
depth_01 -13.815927932333222
depth_01 -16.350552632935347
depth_01 -19.151540835805992
depth_01 -22.24687175218868
depth_01 -25.667459058670023
depth_01 -29.447457087455206
depth_01 -33.62459841385048
depth_01 -38.24056586677453
depth_01 -43.341402211041604
depth_01 -48.97796096317406
depth_01 -55.20640199365842
depth_01 -62.088735720175855
depth_01 -69.69341978534455
depth_01 -78.09601210263148
depth_01 -87.37988400213499
depth_01 -97.63699685168024
depth_01 -108.96874488209758
depth_01 -121.48686590184016
depth_01 -135.31441999260392
depth_01 -150.5868339523135
depth_01 -167.45300594349283
depth_01 -186.07646019149857
depth_01 -206.63653528423634
depth_01 -229.3295811897483
depth_01 -254.3701289430592
depth_01 -281.99198245469614
depth_01 -312.44

In [76]:
temp_ym32_xr = xr.DataArray(np.transpose(temp_ym32.values, (1,2,0)), name='temp_ym32_obs',
                        coords=[lat, lon, year], 
                        dims=['lat', 'lon', 'year'])
print(temp_ym32_xr)
temp_ym32_xr.to_netcdf(output_path + 'temp_ym32_obs.nc')

temp_ym32_slope_xr = xr.DataArray(temp_ym32_slope, name='temp_ym32_slope_obs',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_ym32_slope_xr)
temp_ym32_slope_xr.to_netcdf(output_path + 'temp_ym32_slope_obs.nc')

temp_ym32_p_value_xr = xr.DataArray(temp_ym32_p_value, name='temp_ym32_p_value_obs',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_ym32_p_value_xr)
temp_ym32_p_value_xr.to_netcdf(output_path + 'temp_ym32_p_value_obs.nc')

temp_ym32_std_err_xr = xr.DataArray(temp_ym32_std_err, name='temp_ym32_std_err_obs',
                        coords=[lat, lon], 
                        dims=['lat', 'lon'])
print(temp_ym32_std_err_xr)
temp_ym32_std_err_xr.to_netcdf(output_path + 'temp_ym32_std_err_obs.nc')

NameError: name 'PRESSURE' is not defined

In [77]:
temp_ym32_xr = xr.DataArray(np.transpose(temp_ym32.values, (1,2,0)), name='temp_ym32_obs',
                        coords=[depth, lon, year], 
                        dims=['depth', 'lon', 'year'])
print(temp_ym32_xr)
temp_ym32_xr.to_netcdf(output_path + 'temp_ym32_obs.nc')

temp_ym32_slope_xr = xr.DataArray(temp_ym32_slope, name='temp_ym32_slope_obs',
                        coords=[depth, lon], 
                        dims=['depth', 'lon'])
print(temp_ym32_slope_xr)
temp_ym32_slope_xr.to_netcdf(output_path + 'temp_ym32_slope_obs.nc')

temp_ym32_p_value_xr = xr.DataArray(temp_ym32_p_value, name='temp_ym32_p_value_obs',
                        coords=[depth, lon], 
                        dims=['depth', 'lon'])
print(temp_ym32_p_value_xr)
temp_ym32_p_value_xr.to_netcdf(output_path + 'temp_ym32_p_value_obs.nc')

temp_ym32_std_err_xr = xr.DataArray(temp_ym32_std_err, name='temp_ym32_std_err_obs',
                        coords=[depth, lon], 
                        dims=['depth', 'lon'])
print(temp_ym32_std_err_xr)
temp_ym32_std_err_xr.to_netcdf(output_path + 'temp_ym32_std_err_obs.nc')




temp_ym32_1_xr = xr.DataArray(temp_ym32_1, name='temp_ym32_1_obs',
                        coords=[depth_1, lon_1, year], 
                        dims=['depth_1', 'lon_1', 'year'])
print(temp_ym32_1_xr)
temp_ym32_1_xr.to_netcdf(output_path + 'temp_ym32_1_obs.nc')

temp_ym32_1_slope_xr = xr.DataArray(temp_ym32_1_slope, name='temp_ym32_1_slope_obs',
                        coords=[depth_1, lon_1], 
                        dims=['depth_1', 'lon_1'])
print(temp_ym32_1_slope_xr)
temp_ym32_1_slope_xr.to_netcdf(output_path + 'temp_ym32_1_slope_obs.nc')

temp_ym32_1_p_value_xr = xr.DataArray(temp_ym32_1_p_value, name='temp_ym32_1_p_value_obs',
                        coords=[depth_1, lon_1], 
                        dims=['depth_1', 'lon_1'])
print(temp_ym32_1_p_value_xr)
temp_ym32_1_p_value_xr.to_netcdf(output_path + 'temp_ym32_1_p_value_obs.nc')

temp_ym32_1_std_err_xr = xr.DataArray(temp_ym32_1_std_err, name='temp_ym32_1_std_err_obs',
                        coords=[depth_1, lon_1], 
                        dims=['depth_1', 'lon_1'])
print(temp_ym32_1_std_err_xr)
temp_ym32_1_std_err_xr.to_netcdf(output_path + 'temp_ym32_1_std_err_obs.nc')




temp_ym32_025_xr = xr.DataArray(temp_ym32_025, name='temp_ym32_025_obs',
                        coords=[depth_025, lon_025, year], 
                        dims=['depth_025', 'lon_025', 'year'])
print(temp_ym32_025_xr)
temp_ym32_025_xr.to_netcdf(output_path + 'temp_ym32_025_obs.nc')

temp_ym32_025_slope_xr = xr.DataArray(temp_ym32_025_slope, name='temp_ym32_025_slope_obs',
                        coords=[depth_025, lon_025], 
                        dims=['depth_025', 'lon_025'])
print(temp_ym32_025_slope_xr)
temp_ym32_025_slope_xr.to_netcdf(output_path + 'temp_ym32_025_slope_obs.nc')

temp_ym32_025_p_value_xr = xr.DataArray(temp_ym32_025_p_value, name='temp_ym32_025_p_value_obs',
                        coords=[depth_025, lon_025], 
                        dims=['depth_025', 'lon_025'])
print(temp_ym32_025_p_value_xr)
temp_ym32_025_p_value_xr.to_netcdf(output_path + 'temp_ym32_025_p_value_obs.nc')

temp_ym32_025_std_err_xr = xr.DataArray(temp_ym32_025_std_err, name='temp_ym32_025_std_err_obs',
                        coords=[depth_025, lon_025], 
                        dims=['depth_025', 'lon_025'])
print(temp_ym32_025_std_err_xr)
temp_ym32_025_std_err_xr.to_netcdf(output_path + 'temp_ym32_025_std_err_obs.nc')




temp_ym32_01_xr = xr.DataArray(temp_ym32_01, name='temp_ym32_01_obs',
                        coords=[depth_01, lon_01, year], 
                        dims=['depth_01', 'lon_01', 'year'])
print(temp_ym32_01_xr)
temp_ym32_01_xr.to_netcdf(output_path + 'temp_ym32_01_obs.nc')

temp_ym32_01_slope_xr = xr.DataArray(temp_ym32_01_slope, name='temp_ym32_01_slope_obs',
                        coords=[depth_01, lon_01], 
                        dims=['depth_01', 'lon_01'])
print(temp_ym32_01_slope_xr)
temp_ym32_01_slope_xr.to_netcdf(output_path + 'temp_ym32_01_slope_obs.nc')

temp_ym32_01_p_value_xr = xr.DataArray(temp_ym32_01_p_value, name='temp_ym32_01_p_value_obs',
                        coords=[depth_01, lon_01], 
                        dims=['depth_01', 'lon_01'])
print(temp_ym32_01_p_value_xr)
temp_ym32_01_p_value_xr.to_netcdf(output_path + 'temp_ym32_01_p_value_obs.nc')

temp_ym32_01_std_err_xr = xr.DataArray(temp_ym32_01_std_err, name='temp_ym32_01_std_err_obs',
                        coords=[depth_01, lon_01], 
                        dims=['depth_01', 'lon_01'])
print(temp_ym32_01_std_err_xr)
temp_ym32_01_std_err_xr.to_netcdf(output_path + 'temp_ym32_01_std_err_obs.nc')

<xarray.DataArray 'temp_ym32_obs' (depth: 58, lon: 32, year: 14)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [22.167917 , 22.332167 , 22.104084 , ..., 22.430334 ,
         22.63475  , 22.6065   ],
        ...,
        [19.375    , 19.438583 , 19.52125  , ..., 19.374584 ,
         19.953167 , 19.82625  ],
        [19.330332 , 19.376749 , 19.475582 , ..., 19.231333 ,
         19.878416 , 19.781916 ],
        [19.228249 , 19.242167 , 19.359333 , ..., 19.047583 ,
         19.762    , 19.764334 ]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [22.044582 , 22.206165 , 21.985416 , ..., 22.301832 ,
         22.510834 , 22.478083 ],
        ...,
        [19.347    , 19.409332 , 19.474667

<xarray.DataArray 'temp_ym32_1_slope_obs' (depth_1: 34, lon_1: 32)>
array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ..., 0.02270235, 0.02102779,
        0.02302894],
       [       nan,        nan,        nan, ..., 0.02280986, 0.02085849,
        0.02259245],
       ...,
       [       nan,        nan,        nan, ..., 0.00066779, 0.00058984,
        0.0005064 ],
       [       nan,        nan,        nan, ..., 0.00126307, 0.00129045,
        0.00124474],
       [       nan,        nan,        nan, ..., 0.00109237, 0.00109044,
        0.00093905]])
Coordinates:
  * depth_1  (depth_1) float64 -1.152 -3.649 -6.565 ... -1.67e+03 -1.854e+03
  * lon_1    (lon_1) float64 150.5 151.5 152.5 153.5 ... 178.5 179.5 180.5 181.5
<xarray.DataArray 'temp_ym32_1_p_value_obs' (depth_1: 34, lon_1: 32)>
array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan, 

<xarray.DataArray 'temp_ym32_025_std_err_obs' (depth_025: 34, lon_025: 128)>
array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ..., 0.01382219,        nan,
               nan],
       [       nan,        nan,        nan, ..., 0.01372031,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ..., 0.00100592,        nan,
               nan],
       [       nan,        nan,        nan, ..., 0.000641  ,        nan,
               nan],
       [       nan,        nan,        nan, ..., 0.00042922,        nan,
               nan]])
Coordinates:
  * depth_025  (depth_025) float64 -1.152 -3.649 -6.565 ... -1.67e+03 -1.854e+03
  * lon_025    (lon_025) float64 150.1 150.4 150.6 150.9 ... 181.4 181.6 181.9
<xarray.DataArray 'temp_ym32_01_obs' (depth_01: 54, lon_01: 320, year: 14)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
       